In [9]:
import sys
import os

# Go up one level (from notebooks/ to project root)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from PyPDF2 import PdfReader

def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

In [5]:
FILE_PATH = "../data/Handelsblatt_Artikel_Dobelli.pdf"

In [6]:
load_pdf(FILE_PATH)

'19.04.25, 17:36Geldanlage: „Je kompetenter Sie sind, desto konzentrierter sollte Ihr Depot sein“\nPage 1 of 15https://www.handelsblatt.com/finanzen/anlagestrategie/geldanlage-je…-sie-sind-desto-konzentrierter-sollte-ihr-depot-sein/100105520.htmlGeldanlage„Je kompetenter Sie sind,desto konzentrierter sollte IhrDepot sein“Rolf Dobelli hat viele Gesichter – das des Investorsist weniger bekannt. Hier verrät er erstmals, wie ersein Geld anlegt, wieso er weiter auf die USA setzt– und große Beträge nur bar zahlt.Benjamin Ansari18.04.2025 - 08:17 UhrArtikel anhören24:39\nRolf Dobelli: Mit Ehefrau und Zwillingssöhnen wohnt er in Bern. Foto: IMAGO/Newscom / El Pais\nFrankfurt. Steht ein schwerreicher Philosoph am Stehtisch und lästert über all dieFondsmanager, die sich um ihn scharen. So beginnt kein schlechter Witz, sondernder Nachmittag von Rolf Dobelli.An diesem Frühlingstag Anfang April steht der Schweizer Starautor in blauem Anzugmit Einstecktuch im Congress Center der Frankfurter Messe vo

In [10]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from src.config import OPENAI_API_KEY

embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

def embed_and_store(text, persist_directory=".chroma"):
    docs = splitter.create_documents([text])
    vectordb = Chroma.from_documents(docs, embedding, persist_directory=persist_directory)
    vectordb.persist()
    return vectordb


/var/folders/2x/vcryf4w947j2hgyynh9njl140000gn/T/ipykernel_1461/2874186048.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [11]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from src.config import OPENAI_API_KEY

persist_directory = ".chroma"

def ask(query):
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=None)
    retriever = vectordb.as_retriever()
    qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=OPENAI_API_KEY), retriever=retriever)
    return qa.run(query)

In [13]:
ask("Which company does Dobelli love investing in?")

/var/folders/2x/vcryf4w947j2hgyynh9njl140000gn/T/ipykernel_1461/3565616611.py:11: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  qa = RetrievalQA.from_chain_type(llm=OpenAI(openai_api_key=OPENAI_API_KEY), retriever=retriever)
/var/folders/2x/vcryf4w947j2hgyynh9njl140000gn/T/ipykernel_1461/3565616611.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return qa.run(query)


ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/guides/embeddings in query.